In [1]:
# Excel Scenario Model Generator
# Interactive what-if analysis for founders

import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import LineChart, Reference, BarChart
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("EXCEL SCENARIO MODEL GENERATOR")
print("="*70)

# Load data
df_monthly = pd.read_csv("D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Data/monthly_financials.csv")
df_detailed = pd.read_csv("D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Data/initiative_monthly_detail.csv")
df_initiatives = pd.read_csv("D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Data/initiative_master.csv")

# Get latest month data
latest_month = df_detailed['month'].max()
latest_data = df_detailed[df_detailed['month'] == latest_month].copy()

current_cash = df_monthly.iloc[-1]['cash_balance']
current_runway = df_monthly.iloc[-1]['runway_months']

EXCEL SCENARIO MODEL GENERATOR


In [2]:
# ============================================
# CREATE WORKBOOK
# ============================================

wb = Workbook()

# Define styles
header_fill = PatternFill(start_color="2E75B6", end_color="2E75B6", fill_type="solid")
header_font = Font(bold=True, color="FFFFFF", size=11)
title_font = Font(bold=True, size=14, color="2E75B6")
currency_format = '_($* #,##0.00_);_($* (#,##0.00);_($* "-"??_);_(@_)'
percent_format = '0.0%'

In [3]:
# ============================================
# SHEET 1: SCENARIO BUILDER
# ============================================

ws1 = wb.active
ws1.title = "Scenario Builder"

# Title
ws1['A1'] = "RUNWAY KILL-SWITCH SCENARIO BUILDER"
ws1['A1'].font = Font(bold=True, size=16, color="2E75B6")
ws1.merge_cells('A1:H1')

ws1['A2'] = f"Current Runway: {current_runway:.1f} months | Cash Balance: ${current_cash:,.0f}"
ws1['A2'].font = Font(size=11, italic=True)
ws1.merge_cells('A2:H2')

# Headers
row = 4
ws1[f'A{row}'] = "Initiative"
ws1[f'B{row}'] = "Monthly Burn"
ws1[f'C{row}'] = "Monthly Revenue"
ws1[f'D{row}'] = "Net Burn"
ws1[f'E{row}'] = "Current ROI %"
ws1[f'F{row}'] = "Action"
ws1[f'G{row}'] = "Status After"
ws1[f'H{row}'] = "Runway Impact"

for col in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
    ws1[f'{col}{row}'].font = header_font
    ws1[f'{col}{row}'].fill = header_fill
    ws1[f'{col}{row}'].alignment = Alignment(horizontal='center')

# Data
row = 5
for idx, init_row in latest_data.iterrows():
    init_id = init_row['initiative_id']
    name = init_row['initiative_name']
    burn = init_row['monthly_burn']
    revenue = init_row['monthly_revenue']
    net_burn = burn - revenue
    roi = ((revenue - burn) / burn * 100) if burn > 0 else 0
    
    ws1[f'A{row}'] = name
    ws1[f'B{row}'] = burn
    ws1[f'B{row}'].number_format = currency_format
    ws1[f'C{row}'] = revenue
    ws1[f'C{row}'].number_format = currency_format
    ws1[f'D{row}'] = net_burn
    ws1[f'D{row}'].number_format = currency_format
    ws1[f'E{row}'] = roi / 100
    ws1[f'E{row}'].number_format = percent_format
    
    # Dropdown for Action (in real Excel, you'd add data validation)
    ws1[f'F{row}'] = "KEEP"  # Default
    ws1[f'F{row}'].font = Font(bold=True)
    
    # Status formula (would be dynamic in real Excel)
    ws1[f'G{row}'] = "=IF(F{0}=\"KILL\",\"ELIMINATED\",IF(F{0}=\"PAUSE\",\"PAUSED\",\"ACTIVE\"))".replace("{0}", str(row))
    
    # Runway impact if killed
    new_net_burn = df_monthly.iloc[-1]['net_burn'] - net_burn
    if new_net_burn > 0:
        new_runway = current_cash / new_net_burn
        impact = new_runway - current_runway
    else:
        impact = 999
    
    ws1[f'H{row}'] = impact
    ws1[f'H{row}'].number_format = '0.0'
    
    row += 1

# Summary section
row += 2
ws1[f'A{row}'] = "SCENARIO SUMMARY"
ws1[f'A{row}'].font = title_font
ws1.merge_cells(f'A{row}:H{row}')

row += 1
ws1[f'A{row}'] = "Total Burn (All Initiatives):"
ws1[f'B{row}'] = f"=SUM(B5:B{row-3})"
ws1[f'B{row}'].number_format = currency_format
ws1[f'B{row}'].font = Font(bold=True)

row += 1
ws1[f'A{row}'] = "Total Revenue (All Initiatives):"
ws1[f'B{row}'] = f"=SUM(C5:C{row-4})"
ws1[f'B{row}'].number_format = currency_format
ws1[f'B{row}'].font = Font(bold=True)

row += 1
ws1[f'A{row}'] = "Net Monthly Burn:"
ws1[f'B{row}'] = f"=SUM(D5:D{row-5})"
ws1[f'B{row}'].number_format = currency_format
ws1[f'B{row}'].font = Font(bold=True, color="FF0000")

row += 1
ws1[f'A{row}'] = "Projected Runway:"
ws1[f'B{row}'] = f"={current_cash}/B{row-1}"
ws1[f'B{row}'].number_format = '0.0 "months"'
ws1[f'B{row}'].font = Font(bold=True, size=12, color="2E75B6")

row += 1
ws1[f'A{row}'] = "Runway Extension:"
ws1[f'B{row}'] = f"=B{row-1}-{current_runway}"
ws1[f'B{row}'].number_format = '0.0 "months"'
ws1[f'B{row}'].font = Font(bold=True, size=12, color="00B050")

# Column widths
ws1.column_dimensions['A'].width = 30
ws1.column_dimensions['B'].width = 15
ws1.column_dimensions['C'].width = 15
ws1.column_dimensions['D'].width = 15
ws1.column_dimensions['E'].width = 12
ws1.column_dimensions['F'].width = 12
ws1.column_dimensions['G'].width = 15
ws1.column_dimensions['H'].width = 15

In [4]:
# ============================================
# SHEET 2: RUNWAY PROJECTION
# ============================================

ws2 = wb.create_sheet("Runway Projection")

ws2['A1'] = "RUNWAY PROJECTION - NEXT 24 MONTHS"
ws2['A1'].font = title_font
ws2.merge_cells('A1:E1')

# Headers
ws2['A3'] = "Month"
ws2['B3'] = "Current Trajectory"
ws2['C3'] = "Kill Negative ROI"
ws2['D3'] = "Scale Core SaaS"
ws2['E3'] = "Optimal Scenario"

for col in ['A', 'B', 'C', 'D', 'E']:
    ws2[f'{col}3'].font = header_font
    ws2[f'{col}3'].fill = header_fill

# Calculate projections
current_net_burn = df_monthly.iloc[-1]['net_burn']

# Scenario 1: Current trajectory
# Scenario 2: Kill all negative ROI (saves $119K/month)
# Scenario 3: Scale Core SaaS (adds $56K/month profit)
# Scenario 4: Kill negative + scale positive

kill_savings = 119665  # From Query 12
scale_profit = 56000   # From earlier calculation

for month in range(0, 25):
    row = 4 + month
    ws2[f'A{row}'] = month
    
    # Current trajectory
    cash_current = current_cash - (current_net_burn * month)
    ws2[f'B{row}'] = cash_current if cash_current > 0 else 0
    ws2[f'B{row}'].number_format = currency_format
    
    # Kill negative ROI
    new_burn = current_net_burn - kill_savings
    cash_kill = current_cash - (new_burn * month) if new_burn > 0 else current_cash + (abs(new_burn) * month)
    ws2[f'C{row}'] = cash_kill
    ws2[f'C{row}'].number_format = currency_format
    
    # Scale Core SaaS
    cash_scale = current_cash - ((current_net_burn - scale_profit) * month)
    ws2[f'D{row}'] = cash_scale if cash_scale > 0 else 0
    ws2[f'D{row}'].number_format = currency_format
    
    # Optimal (kill + scale)
    optimal_burn = current_net_burn - kill_savings - scale_profit
    cash_optimal = current_cash - (optimal_burn * month) if optimal_burn > 0 else current_cash + (abs(optimal_burn) * month)
    ws2[f'E{row}'] = cash_optimal
    ws2[f'E{row}'].number_format = currency_format

# Add chart
chart = LineChart()
chart.title = "Cash Balance Projection"
chart.style = 10
chart.y_axis.title = "Cash Balance ($)"
chart.x_axis.title = "Months from Now"

data = Reference(ws2, min_col=2, min_row=3, max_col=5, max_row=28)
cats = Reference(ws2, min_col=1, min_row=4, max_row=28)
chart.add_data(data, titles_from_data=True)
chart.set_categories(cats)

ws2.add_chart(chart, "G3")

# Column widths
for col in ['A', 'B', 'C', 'D', 'E']:
    ws2.column_dimensions[col].width = 18

In [5]:
# ============================================
# SHEET 3: INSTRUCTIONS
# ============================================

ws3 = wb.create_sheet("Instructions")

ws3['A1'] = "HOW TO USE THIS MODEL"
ws3['A1'].font = Font(bold=True, size=14, color="2E75B6")
ws3.merge_cells('A1:D1')

instructions = [
    "",
    "SCENARIO BUILDER SHEET:",
    "1. In column F, change 'KEEP' to 'KILL' or 'PAUSE' for any initiative",
    "2. The model automatically calculates new runway at the bottom",
    "3. Green number shows runway extension (or reduction if negative)",
    "",
    "RUNWAY PROJECTION SHEET:",
    "• Shows cash balance over next 24 months under 4 scenarios",
    "• Blue line: Current trajectory (do nothing)",
    "• Orange line: Kill all negative ROI initiatives",
    "• Gray line: Scale Core SaaS by 50%",
    "• Yellow line: Optimal (kill negatives + scale winner)",
    "",
    "KEY INSIGHTS:",
    "• APAC Expansion is burning $51K/month with $0 revenue → KILL IMMEDIATELY",
    "• Content Marketing: -77% ROI → KILL",
    "• Mobile App: -73% ROI → KILL",
    "• AI Feature: -98% ROI → KILL",
    "• Core SaaS: +38% ROI → SCALE",
    "",
    "DECISION IMPACT:",
    f"• Killing all negative ROI initiatives extends runway by {kill_savings / current_net_burn:.1f} months",
    f"• Monthly cash savings: ${kill_savings:,.0f}",
    f"• Revenue lost: Only ${14641:,.0f} (7.5% of total revenue)",
    "",
    "THIS IS A NO-BRAINER DECISION."
]

row = 3
for instruction in instructions:
    ws3[f'A{row}'] = instruction
    if instruction.startswith("•") or instruction.startswith("DECISION") or instruction.startswith("THIS IS"):
        ws3[f'A{row}'].font = Font(bold=True, size=11)
    row += 1

ws3.column_dimensions['A'].width = 100

In [6]:
# ============================================
# SAVE WORKBOOK
# ============================================

filename = "D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Excel Models/Runway_KillSwitch_Scenario_Model.xlsx"
wb.save(filename)

print(f"✅ Excel model created: {filename}")
print("\nModel includes:")
print("  • Scenario Builder (interactive kill switches)")
print("  • Runway Projection (4 scenarios over 24 months)")
print("  • Instructions (how to use)")
print(f"\n💰 Key Insight: Killing negative ROI initiatives")
print(f"   extends runway from {current_runway:.1f} → 999 months (profitable!)")

✅ Excel model created: D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Excel Models/Runway_KillSwitch_Scenario_Model.xlsx

Model includes:
  • Scenario Builder (interactive kill switches)
  • Runway Projection (4 scenarios over 24 months)
  • Instructions (how to use)

💰 Key Insight: Killing negative ROI initiatives
   extends runway from 21.5 → 999 months (profitable!)
